<a href="https://colab.research.google.com/github/goeludit/langchain-rag-document-analyzer/blob/main/RAG_Langchain_Document_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install langchain_community
!pip install langchain-openai
!pip install --upgrade --quiet  "unstructured[all-docs]"
!pip install langchain-chroma
!pip install langchain-text-splitter
!pip install openai
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.4/360.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6

In [3]:
#Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
os.environ['OPENAI_API_KEY'] = 'Place your OpenAI key here'

In [4]:
#Import all the libraries
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
# from langchain.chat_models import OpenAI
from langchain_community.chat_models.openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate

from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

import os, openai, shutil, getpass

In [5]:
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/RAG Langchain/files' #Path to read the data from the documents (Eg: Markdown)
CHROMA_PATH = "/content/drive/MyDrive/Colab Notebooks/RAG Langchain/Chroma-openai" #Path where the chromadb will store the data

def load_docs():
  loader = DirectoryLoader(DATA_PATH, show_progress= True)
  docs = loader.load()
  return docs

In [6]:
def text_split(docs):

  text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, #Experiment with a chunk size and the chunk overlap
        chunk_overlap=500,
        length_function=len,
        # is_separator_regex=False,
        add_start_index=True)

  chunks = text_splitter.split_documents(docs)
  document = chunks[11]
  print(document.page_content)
  print(document.metadata)
  return chunks

In [7]:

def save_to_chroma(chunks):
  if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

  db = Chroma.from_documents(chunks, embedding=OpenAIEmbeddings(), persist_directory=CHROMA_PATH)
  db.persist()
  print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

In [8]:
documents = load_docs()

100%|██████████| 1/1 [00:15<00:00, 15.38s/it]


In [9]:
chunks = text_split(documents)

"Who better than you to know that?" the goddesses said in wonder. "After all, it was by your banks that he knelt each day to contemplate himself!"

The lake was silent for some time. Finally, it said:\ "I weep for Narcissus, but I never noticed that Narcissus was beautiful. I weep because, each time he knelt beside my banks, I could see, in the depths of his eyes, my own beauty reflected." "What a lovely story," the alchemist thought.

PART ONE**

THE BOY'S NAME WAS SANTIAGO. DUSK WAS FALLING AS the boy arrived with his herd at an abandoned church. The roof had fallen in long ago, and an enormous sycamore had grown on the spot where the sacristy had once stood.
{'source': '/content/drive/MyDrive/Colab Notebooks/RAG Langchain/files/the_alchemist.markdown', 'start_index': 7114}


In [10]:

save_to_chroma(chunks)



Saved 383 chunks to /content/drive/MyDrive/Colab Notebooks/RAG Langchain/Chroma-openai.


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [11]:


# Prepare the DB.
embedding_function = OpenAIEmbeddings()
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding_function)

query_text = "Where was the gold found?"

# Search the DB.
results = db.similarity_search_with_relevance_scores(query_text, k=3)


print(results)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


[(Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/RAG Langchain/files/the_alchemist.markdown', 'start_index': 213545}, page_content='> "There\'s gold here," he said.\n\nThe moon shone on the face of the Arab who had seized him, and in the man\'s eyes the boy saw death.\n\n"He\'s probably got more gold hidden in the ground. "\\ They made the boy continue digging, but he found nothing. As the sun rose, the men began to beat the boy. He was bruised and bleeding, his clothing was torn to shreds, and he felt that death was near.\n\n"What good is money to you if you\'re going to die? It\'s not often that money can save someone\'s life," the alchemist had said. Finally, the boy screamed at the men, "I\'m digging for treasure!" And, although his mouth was bleeding and swollen, he told his attackers that he had twice dreamed of a treasure hidden near the Pyramids of Egypt.'), 0.7819075611153079), (Document(metadata={'source': '/content/drive/MyDrive/Colab Notebooks/RAG Lang

In [12]:
for i in results:
  print(i[0].page_content)
  print('--------------------')


> "There's gold here," he said.

The moon shone on the face of the Arab who had seized him, and in the man's eyes the boy saw death.

"He's probably got more gold hidden in the ground. "\ They made the boy continue digging, but he found nothing. As the sun rose, the men began to beat the boy. He was bruised and bleeding, his clothing was torn to shreds, and he felt that death was near.

"What good is money to you if you're going to die? It's not often that money can save someone's life," the alchemist had said. Finally, the boy screamed at the men, "I'm digging for treasure!" And, although his mouth was bleeding and swollen, he told his attackers that he had twice dreamed of a treasure hidden near the Pyramids of Egypt.
--------------------
"We're refugees from the tribal wars, and we need money," the other figure said. "What are you hiding there? "\ "I'm not hiding anything," the boy answered.

But one of them seized the boy and yanked him back out of the hole. Another, who was search

In [ ]:
PROMPT_TEMPLATE = """

Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

context_text = "\n\n---\n\n".join([doc.page_content for doc, score in results])
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format_messages(context=context_text, question=query_text)

In [ ]:
response_ref = "".join([i.content for i in prompt])

In [ ]:
model = ChatOpenAI()
response_text = model.predict(response_ref)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
response_text

'The gold was found buried at the base of a sycamore tree.'

In [ ]:
# The gold was found buried at the base of a sycamore tree.